# * Sales Data

## Parameter

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  DIM_CORP_KPI

In [2]:
''' Execute Summary Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, PRODUCT, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, METRIC_CD, METRIC_NAME
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summary DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-07, 09:38:33

   -> Ex-T Sales DataFrame : 829 rows, 17 columns

TDMDBPR : Disconnected


In [11]:
''' Gross Adds Summary '''

ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-03-07 08:36:33,20240101,20250305,0,"887,796","860,427","860,427","860,427","860,427","860,427",7,6204,663093
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-03-07 08:36:33,20240101,20250305,0,"141,686","75,956","75,956","75,956","75,956","75,956",7,1528,75567
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-03-07 08:36:33,20240101,20250306,0,"2,167,129","1,461,185","1,461,185","1,461,185","1,461,185","1,461,185",7,2211,415501
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-03-07 08:36:33,20240101,20250306,0,"354,182","119,073","119,073","119,073","119,073","119,073",7,781,60627
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-03-07 08:06:36,20240101,20250305,0,"24,129,731","23,158,009","23,158,009","22,824,123","23,162,365","23,040,502",7,7441,1594851
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-03-07 08:06:36,20240101,20250306,0,"22,626,383","17,504,047","17,504,047","17,026,276","17,504,047","17,502,390",7,5817,1691079
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-03-07 08:00:39,20240102,20250306,0,"663,980","472,537","472,537","465,264","469,709","469,524",7,1174,298948
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-03-07 08:00:39,20240102,20250306,0,"21,967",540,540,524,525,525,7,99,1984
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-03-07 08:00:39,20240101,20250306,0,"911,647","643,230","643,230","631,813","637,553","637,244",7,1184,314370
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-03-07 08:00:39,20240101,20250306,0,"49,116","15,522","15,522","15,332","15,605","15,605",7,620,25479


In [13]:
''' Inflow Summary '''

inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-03-07 08:36:33,20240101,20250305,0,"384,470,027","375,604,667","375,604,667","375,604,667","375,604,667","375,604,667",7,6204,663093
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-03-07 08:36:33,20240101,20250305,0,"139,250,810","25,964,937","25,964,937","25,964,937","25,964,937","25,964,937",7,1528,75567
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-03-07 08:36:33,20240101,20250306,0,"908,597,593","698,862,769","698,862,769","698,862,769","698,862,769","698,862,769",7,2211,415501
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-03-07 08:36:33,20240101,20250306,0,"80,088,949","33,798,732","33,798,732","33,798,732","33,798,732","33,798,732",7,781,60627
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-03-07 08:06:36,20240101,20250305,0,"7,762,372,837","7,618,103,106","7,618,103,106","7,492,381,468","7,619,387,182","7,572,533,670",7,7418,4735354
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-03-07 08:06:36,20240201,20250305,0,"1,205,560,366","1,186,703,496","1,186,703,496","1,171,574,409","1,186,966,946","1,175,023,260",7,7294,2589797
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-03-07 08:06:36,20240101,20250304,0,"6,489,189,049","5,491,288,730","5,491,288,730","5,314,729,078","5,491,288,730","5,490,852,208",7,5793,3936157
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-03-07 08:06:36,20240101,20250304,0,"1,293,399,807","1,279,353,514","1,279,353,514","1,251,745,154","1,279,353,514","1,279,165,999",7,5739,2136261
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-03-07 08:00:39,20240102,20250306,0,"366,764,359","265,994,660","265,994,660","261,736,550","264,482,319","264,372,676",7,1174,298948
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-03-07 08:00:39,20240102,20250306,0,"26,975,225","1,731,884","1,731,884","1,697,097","1,700,196","1,700,196",7,99,1984


In [7]:
''' METRIC Summary '''

src_t_sales_mth_df = chk_src_df.copy()
src_t_sales_mth_df = src_t_sales_mth_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_t_sales_mth_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_sales_mth_df.to_string(max_cols=10)}')


     PRODUCT       METRIC_CD                                                             METRIC_NAME           LOAD_DATE  START_DAY   END_DAY              P  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   POSTPAID    DB2R010500CG                             Postpaid Inflow M1 B2C : DTAC - GEO Channel 2025-03-07 08:36:33   20240101  20250305    149,486,001              7          798   210019
1   POSTPAID    DB2R010500CS                                           Postpaid Inflow M1 B2C : DTAC 2025-03-07 08:36:33   20240101  20250305    384,470,027              7         6204   663093
2   POSTPAID    DB2R020500CG                             Postpaid Inflow M1 B2B : DTAC - GEO Channel 2025-03-07 08:36:33   20240101  20250305      6,046,478              7          311    23020
3   POSTPAID    DB2R020500CS                                           Postpaid Inflow M1 B2B : DTAC 2025-03-07 08:36:33   20240101  20250305    139,250,810              7         1528    75567
4   POSTPAID    DB2S010100CG 

In [8]:
''' METRIC by Month '''

v_tm_key_mth = '202502'
src_t_sales_mth_df_cols = ['PRODUCT', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

src_t_sales_mth_df = chk_src_df[src_t_sales_mth_df_cols].copy()
src_t_sales_mth_df = src_t_sales_mth_df.loc[src_t_sales_mth_df['TM_KEY_MTH']==v_tm_key_mth]
src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

mod_col_list = src_t_sales_mth_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_sales_mth_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_mth: 202502

     PRODUCT       METRIC_CD                                                             METRIC_NAME           LOAD_DATE TM_KEY_MTH  START_DAY   END_DAY  C            P            G            H           HH         CCAA       CCAATT
0   POSTPAID    DB2R010500CG                             Postpaid Inflow M1 B2C : DTAC - GEO Channel 2025-03-07 08:36:33     202502   20250201  20250228  0    7,014,205    6,890,568    6,890,568    6,890,568    6,890,568    6,890,568
1   POSTPAID    DB2R010500CS                                           Postpaid Inflow M1 B2C : DTAC 2025-03-07 08:36:33     202502   20250201  20250228  0   17,946,445   17,688,096   17,688,096   17,688,096   17,688,096   17,688,096
2   POSTPAID    DB2R020500CG                             Postpaid Inflow M1 B2B : DTAC - GEO Channel 2025-03-07 08:36:33     202502   20250201  20250228  0      382,012      360,044      360,044      360,044      360,044      360,044
3   POSTPAID   

## Actual : Transaction
->  DIM_CORP_KPI

In [5]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
    FROM CDSAPPO.DIM_CORP_KPI
    --WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    WHERE METRIC_CD = 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101

TDMDBPR : Connected

   -> Execute query... 2025-03-07, 18:16:31

   -> Daily DataFrame : 65 rows, 12 columns

TDMDBPR : Disconnected


In [15]:
''' METRIC by Day '''

v_tm_key_day = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

src_sale_day_df = chk_src_df.copy()
src_sale_day_df = src_sale_day_df.loc[src_sale_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_sale_day_df = src_sale_day_df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_sale_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_sale_day_df[col] = src_sale_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_sale_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_sale_day_df.to_string(max_cols=12)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_sale_day_df...

   -> DataFrame : 65 rows, 12 columns


In [17]:
''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

v_metric_cd = 'TB3S000102CS'
v_tm_key_day = 20250201
print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3S000102CS_day_df = chk_src_df.copy()
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['METRIC_CD']==v_metric_cd]
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3S000102CS_day_df = TB3S000102CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3S000102CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3S000102CS_day_df[col] = TB3S000102CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3S000102CS_day_df


TB3S000102CS : TOL Gross Adds Connected : Consumer...

   -> v_metric_cd: TB3S000102CS
   -> v_tm_key_day: 20250201


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250201,2025-03-07 16:45:57,0,"1,608","1,180","1,180","1,167","1,180","1,179"
1,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250202,2025-03-07 16:45:57,0,"1,550","1,141","1,141","1,136","1,141","1,141"
2,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250203,2025-03-07 16:45:57,0,"1,661","1,304","1,304","1,289","1,304","1,303"
3,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250204,2025-03-07 16:45:57,0,"1,629","1,349","1,349","1,335","1,349","1,349"
4,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250205,2025-03-07 16:45:57,0,"1,664","1,359","1,359","1,347","1,359","1,358"
5,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250206,2025-03-07 16:45:57,0,"1,630","1,322","1,322","1,312","1,322","1,321"
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250207,2025-03-07 16:45:57,0,"1,644","1,311","1,311","1,299","1,311","1,310"
7,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250208,2025-03-07 16:45:57,0,"1,813","1,335","1,335","1,326","1,335","1,335"
8,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250209,2025-03-07 16:45:57,0,"1,698","1,263","1,263","1,254","1,263","1,263"
9,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250210,2025-03-07 16:45:57,0,"1,625","1,316","1,316","1,300","1,316","1,316"
